# tests to retrieve the openEO processed habitat map via UDP
This scripts shows how to paramaterize the UDP to retrieve the habitat map from openEO for given year, area. <br>
<br>

In [1]:
import openeo
import os
from eo_processing.utils import laea20km_id_to_extent
from eo_processing.utils.onnx_model_utilities import get_name_output_cube_features
from eo_processing.utils.storage import WEED_storage
from time import sleep

In [2]:
# additional imports for visualization and checks
import rasterio
from rasterio.plot import show

In [3]:
# establish connection to OpenEO and authenticate
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [4]:
terrascope_username = 'deroob' # needed for the Vault access (access to GDrive and S3)
# Note: we are using the standard entry point: "1k27bitdRp41AtHq1xupyqwKaTLzrMUMu"  which represent the openEO folder under WEEDE/Working/WP4_ToolboxDvlpt/openeo_tests
weed_storage = WEED_storage(username=terrascope_username)

Please enter your password for the Terrascope VAULT:  ········


In [125]:
#namespace="/home/deroob/Git/OpenEO-UDP-UDF-catalogue/UDP/json/udp_inference_module_alpha1_prototype.json"
s3_model_directory = 'models'
model_urls = weed_storage.get_onnx_urls(s3_directory=s3_model_directory)
namespace="https://raw.githubusercontent.com/ESA-WEED-project/OpenEO-UDP-UDF-catalogue/refs/heads/alpha1_udp/UDP/json/udp_inference_module_alpha1.json"

In [118]:
#define parameters
param_name = "E468N306"
param_bbox = laea20km_id_to_extent(param_name)
param_year = 2021
param_onnx_models = model_urls
param_output_band_names = get_name_output_cube_features(model_urls)
param_digitalId = "tests/Bert"
param_scenarioId = "CopperNeckMountains"


Loaded ONNX model from /tmp/cache/Level1_class-0_129predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level2_class-C_71predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level2_class-D_68predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level2_class-E_85predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level2_class-F_90predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level2_class-G_164predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level2_class-H_65predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level2_class-I_50predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level2_class-J_62predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level2_class-X_54predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level3_class-C1_62predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level3_class-C3_62predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level3_class-E1_50predictors_v1.onnx
Loaded ONNX model from /tmp/cache/Level3_class-E2_50predictors_v1.onnx
Loaded ONNX mo

## get the habitat map

In [129]:
#get cube from udp
cube = connection.datacube_from_process(
    process_id="udp_inference_module_alpha1",
    namespace=namespace,
    area_name = param_name,
    bbox = param_bbox,
    year = param_year,
    onnx_models = param_onnx_models,
    output_band_names = param_output_band_names,
    digitalId = param_digitalId,
    scenarioId = param_scenarioId
    )

In [130]:
#create and start job
job = cube.create_job(title=f'UDP_tests_{param_digitalId}_{param_scenarioId}_{param_name}',
                                    description=f'inference for tile {param_name}')
job.start_job()

<BatchJob job_id='j-250226160954434ba83c09ee5a257650'>

In [131]:
#follow-up on job
print(job.job_id)
while job.status() not in ['finished','error','canceled']:
    print(f"going to sleep job not yet done: status : {job.status()}")
    sleep(60)

print(f"Job done: status : {job.status()}")

j-250226160954434ba83c09ee5a257650
going to sleep job not yet done: status : queued
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep job not yet done: status : running
going to sleep j

In [124]:
#get results (metadata)
job.get_results()

<JobResults for job 'j-25022615054740b3b2bff0908237fd89'>

In [114]:
#get errors
job.logs(level='error')

[{'id': '[1740570083213, 449579]',
  'time': '2025-02-26T11:41:23.213Z',
  'level': 'error',
  'message': 'Task 19 in stage 77.0 failed 4 times; aborting job'},
 {'id': '[1740570083226, 985923]',
  'time': '2025-02-26T11:41:23.226Z',
  'level': 'error',
  'message': 'Stage error: Job aborted due to stage failure: Task 19 in stage 77.0 failed 4 times, most recent failure: Lost task 19.3 in stage 77.0 (TID 25790) (10.42.221.253 executor 6): java.io.IOException: load_collection/load_stac: error while reading from: /vsis3/EODATA/Sentinel-2/MSI/L2A_N0500/2021/08/28/S2A_MSIL2A_20210828T100031_N0500_R122_T33UWS_20230122T143404.SAFE/GRANULE/L2A_T33UWS_A032294_20210828T100408/IMG_DATA/R20m/T33UWS_20210828T100031_B12_20m.jp2. Detailed error: Unable to parse projection as CRS. GDAL Error Code: 4\n\tat org.openeo.geotrellis.layers.FileLayerProvider$.$anonfun$loadPartitionBySource$1(FileLayerProvider.scala:748)\n\tat scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)\n\tat scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)\n\tat org.apache.spark.shuffle.sort.UnsafeShuffleWriter.write(UnsafeShuffleWriter.java:179)\n\tat org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)\n\tat org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:104)\n\tat org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:54)\n\tat org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:166)\n\tat org.apache.spark.scheduler.Task.run(Task.scala:141)\n\tat org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)\n\tat org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)\n\tat org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)\n\tat org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:94)\n\tat org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:623)\n\tat java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)\n\tat java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)\n\tat java.base/java.lang.Thread.run(Thread.java:829)\nCaused by: geotrellis.raster.gdal.MalformedProjectionException: Unable to parse projection as CRS. GDAL Error Code: 4\n\tat geotrellis.raster.gdal.GDALDataset$.$anonfun$crs$1(GDALDataset.scala:293)\n\tat geotrellis.raster.gdal.GDALDataset$.$anonfun$crs$1$adapted(GDALDataset.scala:290)\n\tat geotrellis.raster.gdal.GDALDataset$.errorHandler$extension(GDALDataset.scala:422)\n\tat geotrellis.raster.gdal.GDALDataset$.crs$extension1(GDALDataset.scala:290)\n\tat geotrellis.raster.gdal.GDALDataset$.crs$extension0(GDALDataset.scala:282)\n\tat geotrellis.raster.gdal.GDALRasterSource.crs$lzycompute(GDALRasterSource.scala:84)\n\tat geotrellis.raster.gdal.GDALRasterSource.crs(GDALRasterSource.scala:84)\n\tat org.openeo.geotrellis.layers.ValueOffsetRasterSource.crs(ValueOffsetRasterSource.scala:93)\n\tat geotrellis.raster.RasterSource.reproject(RasterSource.scala:54)\n\tat org.openeo.geotrellis.layers.BandCompositeRasterSource.$anonfun$reprojectedSources$1(FileLayerProvider.scala:98)\n\tat cats.data.NonEmptyList.map(NonEmptyList.scala:109)\n\tat org.openeo.geotrellis.layers.BandCompositeRasterSource.reprojectedSources(FileLayerProvider.scala:98)\n\tat org.openeo.geotrellis.layers.BandCompositeRasterSource.readBounds(FileLayerProvider.scala:151)\n\tat org.openeo.geotrellis.layers.FileLayerProvider$.$anonfun$loadPartitionBySource$1(FileLayerProvider.scala:746)\n\t... 16 more\n\nDriver stacktrace:'},
 {'id': '[1740570083228, 30944]',
  'time': '2025-02-26T11:41:23.228Z',
  'level': 'error',
  'message': 'Stage error: Job aborted due to stage failure: Task 19 in stage 77.0 failed 4 times, most recent failure: Lost task 19.3 in stage 77.0 (TID 25790) (10.42.221.253 executor 6): java.io.IOException: load_collection/load_stac: error while reading from: /vsis3/EODATA/Sentinel-2/MSI/L2A_N0500/2021/08